In [1]:
import nest_asyncio; nest_asyncio.apply()
import stan
from age_model import get_data

truncated_code = """
data {
  int<lower=1> M; // number of countries
  int<lower=1> N2; // days of observed data + # of days to forecast
  int<lower=1, upper=N2> N_IMP; // number of impact invervention time effects
}
parameters {
  real<lower=0> upswing_timeeff_reduced[N_IMP,M];
  real<lower=0> sd_upswing_timeeff_reduced;
}
model {
  target += normal_lpdf( to_array_1d(upswing_timeeff_reduced[2:N_IMP,:]) | to_array_1d(upswing_timeeff_reduced[1:(N_IMP-1),:]), sd_upswing_timeeff_reduced);
}
"""

data = get_data()
truncated_data = {k: v for k, v in data.items() if k in ["M", "N2", "N_IMP"]}

stan_model = stan.build(truncated_code, data=truncated_data)
stan_fit = stan_model.sample(num_chains=1, num_warmup=20, stepsize=1e-3, max_depth=5, num_samples=2)
params = stan_fit.to_frame().to_dict()

Building...
Found model in cache. Done.
Sampling...
  0/22 [>---------------------------]   0%  1 sec/0     
 22/22 [============================] 100%  1 sec/0   Messages received during sampling:
  Gradient evaluation took 6.8e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.68 seconds.
  Adjust your expectations accordingly!
           three stages of adaptation as currently configured.
           Reducing each adaptation stage to 15%/75%/10% of
           the given number of warmup iterations:
             init_buffer = 3
             adapt_window = 15
             term_buffer = 2

 22/22 [============================] 100%  1 sec/0     
Done.


In [4]:
params

{'lp__': {0: -64.86008126351412, 1: -60.28787050580138},
 'accept_stat__': {0: 0.9806980245959045, 1: 0.9999577083206175},
 'stepsize__': {0: 0.043184714210807024, 1: 0.043184714210807024},
 'treedepth__': {0: 5.0, 1: 5.0},
 'n_leapfrog__': {0: 31.0, 1: 31.0},
 'divergent__': {0: 0.0, 1: 0.0},
 'energy__': {0: 91.9885468853933, 1: 87.84960453727481},
 'upswing_timeeff_reduced.1.1': {0: 2.375799222800284, 1: 1.9291999902099422},
 'upswing_timeeff_reduced.2.1': {0: 2.0841188848600574, 1: 2.666816788370434},
 'upswing_timeeff_reduced.3.1': {0: 2.340021644709324, 1: 2.260486077031473},
 'upswing_timeeff_reduced.4.1': {0: 0.210022177744121, 1: 0.5331445264822792},
 'upswing_timeeff_reduced.5.1': {0: 1.3185738630305759, 1: 1.089362305155561},
 'upswing_timeeff_reduced.6.1': {0: 1.020626988387528, 1: 0.5110392330904157},
 'upswing_timeeff_reduced.7.1': {0: 0.6110298956828462,
  1: 0.42935770482944235},
 'upswing_timeeff_reduced.8.1': {0: 1.687580054379362, 1: 1.0105258315295007},
 'upswing_ti

In [5]:
z0

{'upswing_timeeff_reduced': array([[2.37579922, 3.31418636, 2.30230093, 1.18764866],
        [2.08411888, 3.51511788, 0.57254731, 0.3816605 ],
        [2.34002164, 2.11394295, 0.00855039, 2.46064922],
        [0.21002218, 1.00578725, 0.27990998, 1.68729451],
        [1.31857386, 1.53969438, 0.64451967, 0.99570667],
        [1.02062699, 1.41953321, 1.35763649, 2.69735557],
        [0.6110299 , 1.3071314 , 0.43730879, 2.01413122],
        [1.68758005, 0.64951848, 0.68379999, 2.07862843],
        [1.26821858, 1.71812801, 1.70296433, 1.19259283],
        [3.29274482, 1.19923786, 3.15956383, 2.01411558],
        [2.08607629, 0.23719805, 4.53046059, 3.77720902],
        [0.31710576, 0.79686524, 5.98455141, 4.03460879],
        [1.43268659, 0.35145763, 4.6406972 , 3.91660361],
        [2.48500992, 0.09680731, 4.83495389, 1.59425705]]),
 'sd_upswing_timeeff_reduced': array(1.17960569)}

In [6]:
z1

{'upswing_timeeff_reduced': array([[1.92919999, 5.0557087 , 1.17292306, 2.02269106],
        [2.66681679, 3.80294922, 0.35718699, 0.25458765],
        [2.26048608, 3.20267899, 0.01025214, 2.06039376],
        [0.53314453, 1.99346329, 0.27484261, 1.39043257],
        [1.08936231, 1.27011706, 0.52004219, 1.09441024],
        [0.51103923, 1.83301023, 1.02736699, 1.68943698],
        [0.4293577 , 0.97134196, 0.51920354, 2.83810413],
        [1.01052583, 0.95870184, 1.94241707, 2.5737568 ],
        [1.59707785, 2.08451257, 1.37234592, 0.88653817],
        [1.78404368, 2.03441034, 3.06870575, 1.27653332],
        [1.76988598, 0.17142866, 4.00734679, 2.89767392],
        [0.33193478, 0.46445109, 4.65577193, 3.53939674],
        [1.7428582 , 0.20515866, 6.27045698, 2.42609583],
        [3.44810382, 0.15019801, 6.56376705, 2.90641483]]),
 'sd_upswing_timeeff_reduced': array(0.99378543)}

In [2]:
import numpy as np
import numpyro; numpyro.enable_x64()
import numpyro.distributions as dist

z0 = {}
z1 = {}
for k in stan_fit.param_names:
    if k not in ["upswing_timeeff_reduced", "sd_upswing_timeeff_reduced"]:
        continue
    z0[k] = np.array([v[0] for k1, v in params.items() if k1.startswith(k)])
    z1[k] = np.array([v[1] for k1, v in params.items() if k1.startswith(k)])

    if k == "upswing_timeeff_reduced":
        shape = (data["N_IMP"], data["M"])
        z0[k] = z0[k].reshape((shape[1], shape[0])).T
        z1[k] = z1[k].reshape((shape[1], shape[0])).T
    else:
        z0[k] = z0[k].reshape(())
        z1[k] = z1[k].reshape(())
    # z0[k] = np.log(z0[k])
    # z1[k] = np.log(z1[k])
pe0 = params["lp__"][0]
pe1 = params["lp__"][1]
print("stan pe0 and pe1:", pe0, pe1)
print("stan pe diff:", pe1 - pe0)

stan pe0 and pe1: -64.86008126351412 -60.28787050580138
stan pe diff: 4.572210757712739


In [3]:
npe0 = dist.TruncatedDistribution(
    dist.Normal(z0["upswing_timeeff_reduced"][:-1], z0["sd_upswing_timeeff_reduced"]), low=0.
).log_prob(
    z0["upswing_timeeff_reduced"][1:]
).sum() + np.log(z0["upswing_timeeff_reduced"]).sum() + np.log(z0["sd_upswing_timeeff_reduced"])
npe1 = dist.TruncatedDistribution(
    dist.Normal(z1["upswing_timeeff_reduced"][:-1], z1["sd_upswing_timeeff_reduced"]), low=0.
).log_prob(
    z1["upswing_timeeff_reduced"][1:]
).sum() + np.log(z1["upswing_timeeff_reduced"]).sum() + np.log(z1["sd_upswing_timeeff_reduced"])
print("numpyro pe0 and pe1:", npe0, npe1)
print("numpyro pe diff:", npe1 - npe0)

numpyro pe0 and pe1: -55.86458776348515 -52.22895307311466
numpyro pe diff: 3.6356346903704875
